# **Natural Language Processing Homework 2: Word Sense Ambiguitous (WSD)**

*Authors:*

*   Lorenzo Ciarpaglini (student ID: 1813738)

This is the notebook running my implementation of the WSD task.

# Preliminary

Run this section to import and/or download the libraries needed for the proper functioning of the notebook.

In [ ]:
!git clone https://github.com/SapienzaNLP/nlp2023-hw2

Cloning into 'nlp2023-hw2'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 29 (delta 3), reused 2 (delta 2), pack-reused 20
Receiving objects: 100% (29/29), 11.47 MiB | 15.26 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [ ]:
!pip install gensim
!pip install nltk
!pip install sklearn
!pip install seqeval
!pip install sentencepiece # used by huggingface
!pip install transformers # install huggingface transformers

!pip install gdown==4.6.0

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 440.4 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=f96ac529efaff7e206822d77163f1fdac5a3935cd1b832fc6b335fdd53f25d48
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.0 MB/s eta 0:00:00
  Attempting uninstall: gdo

In [ ]:
!gdown 1aTHMsGote25X48Kk85RGY7LD7UiVPEhk
!gdown 1mb3w8Pk6n74R4avHdgoHIKlRwuIMbYXW

Downloading...
From: https://drive.google.com/uc?id=1aTHMsGote25X48Kk85RGY7LD7UiVPEhk
To: /content/vocabulary_fine.json
100% 1.05M/1.05M [00:00<00:00, 94.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1mb3w8Pk6n74R4avHdgoHIKlRwuIMbYXW
To: /content/vocabulary_coarse.json
100% 963k/963k [00:00<00:00, 106MB/s]


In [ ]:
7#import libraries here
import os
import copy
import math
import gc
import random
import numpy as np

import torch

import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader, random_split

from tqdm import tqdm

import math


from sklearn.metrics import f1_score

import matplotlib.pyplot as plt
from typing import List #, Union, Set, Callable

from transformers import AutoTokenizer, AutoModel, BertForTokenClassification

import json

from seqeval.metrics import accuracy_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score
from seqeval.metrics import precision_score
from seqeval.metrics import recall_score

from sklearn.metrics import f1_score


import tarfile
import zipfile

from google.colab import drive
from google.colab import files as gg_files
drive.mount('/content/drive/')

SEED:int = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True



Mounted at /content/drive/


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
# model_bert_context = AutoModel.from_pretrained("bert-base-cased")
# model_bert_gloss = AutoModel.from_pretrained("bert-base-cased")

In [ ]:
#some global parameters & constants
DATASET_DIR = "nlp2023-hw2/data"
PRINT_BAR = '-' * 10

file_type_dir = "json"  #or "xml"

In [ ]:
def extract_file(path, to_directory='.'):
    if path.endswith('.zip'):
        opener, mode = zipfile.ZipFile, 'r'
    elif path.endswith('.tar.gz') or path.endswith('.tgz'):
        opener, mode = tarfile.open, 'r:gz'
    elif path.endswith('.tar.bz2') or path.endswith('.tbz'):
        opener, mode = tarfile.open, 'r:bz2'
    else:
        print("Error")
        # raise ValueError, "Could not extract `%s` as no appropriate extractor is found" % path

    cwd = os.getcwd()
    os.chdir(to_directory)

    try:
        file = opener(path, mode)
        try: file.extractall()
        finally: file.close()
    finally:
        print("Finally")
        os.chdir(cwd)

In [ ]:
extract_file("data.tar.gz", "nlp2023-hw2/data")

Finally


# DatasetDownloader

In [ ]:
class DatasetDownloader():
    '''
      The DatasetDownloader class set all imported vocabulary pre-created, or creates it from scratch, to map lemmas, pos and tags to their relative indices.
      Sets also the glossary retrieved from the coarse_fine_defs_map.json file needed to map clusters to fine grained.
      Creates a mapping between the original sentence words to the tokens obtained by a tokenizer for a transformer.
      Creates the candidates mask needed to filter out only the target word to be disambiguated and their possible candidates.
    '''
    def __init__(self):
        self.sentences = []
        self.data_set = []
        self.glossary = {}
        self.map_grained_to_fine = {}
        self.map_fine_to_grained = {}

        self.lemmas2idx = {}
        self.tags2idx = {}
        self.pos2idx = {}

        self.idx2lemmas = {}
        self.idx2tags = {}
        self.idx2pos = {}

        self.bep_dicts = []

    def getDictionary(self) -> Tuple[Dict, Dict, Dict, Dict, Dict, Dict]:
        '''
        Return all the dictionaries needed to map all the lemmas, pos and tags to indices and vice-versa
        '''
        return self.lemmas2idx,self.tags2idx,self.pos2idx,self.idx2lemmas,self.idx2tags, self.idx2pos

    def setDictionary(self,
                      lemmas2idx: Dict,
                      tags2idx: Dict,
                      pos2idx: Dict,
                      idx2lemmas: Dict,
                      idx2tags: Dict,
                      idx2pos: Dict) -> None:
        '''
        Set all the dictionaries needed to map all the lemmas, pos and tags to indices and vice-versa
        '''

        self.lemmas2idx = lemmas2idx
        self.tags2idx = tags2idx
        self.pos2idx = pos2idx
        self.idx2lemmas = idx2lemmas
        self.idx2tags = idx2tags
        self.idx2pos = idx2pos

        return

    def setGlossary(self, glossary: Dict) -> None:
        '''
          Set the glossary, which is the mapping between the clusters and the Ground truth
        '''

        self.glossary = glossary

        return

    def getBertDictionary(self) -> None:
        '''
        Return an array mapping for each sentences all words from the original words to the tokens generated by a transformer tokenizer
        '''
        return self.bep_dicts

    def set_data(self,
                 DATASET_DIR: str,
                 DATASET_PREFIX: str,
                 DATASET_FILE: str,
                 FILE_TYPE: str,
                 MAX_LENGTH: int)  -> None:

        '''
        Download the dataset and remove all the sentences with a length greater than MAX_LENGTH, because sentences too big increase too much the batch size.
        '''

        self.downloadDataset(DATASET_DIR, DATASET_PREFIX, DATASET_FILE, FILE_TYPE)
        self.removeWordLongerThan(MAX_LENGTH)
        return

    def set_data_from_sentences(self,
                 sentences: str,
                 MAX_LENGTH: int)  -> None:

        '''
        Download the dataset and remove all the sentences with a length greater than MAX_LENGTH.
        '''

        self.getDatasetFromSentences(sentences)
        self.removeWordLongerThan(MAX_LENGTH)
        return

    def downloadMapGrainedFine(self, DATASET_DIR: str) -> None:
        '''
          Download all the coarse to fine mapping, and create also an inverse map Dict
        '''
        data_path = os.path.join(DATASET_DIR, 'map', 'coarse_fine_defs_map.json')
        with open(data_path) as f:
            dictionary = f.read()
            dictionary = json.loads(dictionary)

            self.map_grained_to_fine = dictionary

        for sense_cluster, senses in self.map_grained_to_fine.items():
            for idx_sense, sense in enumerate(senses):
                for key, value in sense.items():
                    if key not in self.map_fine_to_grained:
                        self.map_fine_to_grained[key] = {
                            'cluster': sense_cluster,
                            'definition':  value
                        }
        return

    def downloadDataset(self, DATASET_DIR: str, DATASET_PREFIX: str, DATASET_FILE: str, FILE_TYPE: str) -> None:
        '''
        Create and array with all elements are dictionaries taken from the file.
        We also add the id, e.g 'id' = "d011.s152"
        '''
        data_path = os.path.join(DATASET_DIR, DATASET_PREFIX)
        data_path = os.path.join(data_path, DATASET_FILE)
        data_path += '.' + FILE_TYPE

        with open(data_path) as f:
            dictionary = f.read()
            dictionary = json.loads(dictionary)

            for idx, value in dictionary.items():
              value['id'] = idx
              self.sentences.append(value)

        return

    def getDatasetFromSentences(self, sentences: List[str]) -> None:
        '''
        Create and array with all elements are dictionaries taken from the file.
        We also add the id, e.g 'id' = "d011.s152"
        '''

        self.sentences = sentences
        return

    def getLongestAndAvgSentence(self) -> Tuple[int, int, float]:
        '''
        Compute the longest and average length of all sentences
        '''
        longest, idx_longest, avg = 0, 0, 0

        for i, data in enumerate(self.sentences):
            if longest < len(data['words']):
                longest = len(data['words'])
                idx_longest = i
            avg += len(data['words'])

        return longest, idx_longest, avg/(i+1)

    def countWordsLongerAvg(self, avg: int) -> int:
        '''
        Count the number of sentences greater than the average length of all sentences
        '''
        count = 0
        for i, data in enumerate(self.sentences):
            if avg < len(data['words']):
                count += 1
        return count

    def removeWordLongerThan(self, max_length: int) -> None:
        '''
        Remove all the sentences longer than max_length
        '''
        sentences_no_longer = []
        for i, data in enumerate(self.sentences):
            if max_length > len(data['words']):
                sentences_no_longer.append(data)

        self.sentences = sentences_no_longer
        return

    def retrieveGlossaryFromCandidates(self) -> None:
        '''
        Get all the senses definitions for all the senses of the correct cluster
        for each word to disambiguate
        '''

        for index, data in tqdm(enumerate(self.sentences)):
            # data = json.loads(data)
            glosses = {}

            for key, candidates in data['candidates'].items():
                glosses[key] = []

                for candidate in candidates:

                    if candidate not in self.glossary.map_fine_to_grained:
                        print(f'candidate {candidate} not present in the mapping')
                        continue

                    #Retrieve the relative sense definition
                    sense = self.glossary.map_fine_to_grained[candidate]
                    glosses[key].append(sense['definition'])

                self.sentences[index]['glosses'] = glosses
        return

    def retrieveGlossaryFromSenses(self) -> None:
        '''
        Get all the senses definitions for all the senses of the correct cluster
        for each word to disambiguate
        '''

        for index, data in tqdm(enumerate(self.sentences)):
            # data = json.loads(data)
            glosses = {}

            for key, senses in data['senses'].items():
                glosses[key] = []

                for sense in senses: #it's just 1

                    if sense not in self.glossary.map_grained_to_fine:
                        print(f'Sense {sense} not present in the mapping')
                        continue

                    #Alternative with alldictionary elements
                    # #Retrieve the relative sense definition
                    # for idx_sense, sense_def in enumerate(self.map_grained_to_fine[sense]):
                    #     glosses[key].append(sense_def)

                    #Retrieve the relative sense definition
                    for idx_sense_arr, sense_def_dict in enumerate(self.glossary.map_grained_to_fine[sense]):
                        for idx_sense, sense_def in sense_def_dict.items():
                            glosses[key].append(sense_def)

                self.sentences[index]['glosses'] = glosses
        return

    def createVocabolary(self) -> None:
        '''
        We create a dictionary(word: int) with all the tokens present all the sentences(a vocabulary)
        for both the lemmas and the tags(candidates, e.g. "snarl.v.h.01").
        Then we add also 'PAD' and 'UNK' tokens in both the vocabularies.
        Finally we create an inverse mapping (int: word), so given the index we can get the word
        '''
        # words_set = set()
        words_set = []
        tags_set = []

        for index, data in tqdm(enumerate(self.sentences)):
            # data = json.loads(data)

            for token in data['lemmas']:
              # words_set.add(token)
              if token not in words_set:
                words_set.append(token)

            for key, candidates in data['candidates'].items():
              # words_set.add(token)
              for candidate in candidates:

                if candidate not in tags_set:
                  tags_set.append(candidate)

        self.lemmas2idx['PAD'] = 0
        self.lemmas2idx['UNK'] = 1

        for idx, word in enumerate(words_set):
            self.lemmas2idx[word] = len(self.lemmas2idx)

        # vocab['UNK'] = idx + 1
        # vocab['PAD'] = idx + 2

        self.tags2idx['PAD'] = 0
        self.tags2idx['UNK'] = 1

        for idx, tag in enumerate(tags_set):
            self.tags2idx[tag] = len(self.tags2idx)

        for key, value in self.lemmas2idx.items():
            self.idx2lemmas[value] = key

        for key, value in self.tags2idx.items():
            self.idx2tags[value] = key

        return

    def createVocabolaryPos(self) -> None:
        '''
        We create a dictionary(tag: int) with all the POS tags(a vocabulary)
        Then we add also 'PAD' token.
        We create also an inverse mapping (int: tag).
        '''
        # words_set = set()
        pos_set = []

        for index, data in tqdm(enumerate(self.sentences)):
        # data = json.loads(data)

            for pos in data['pos_tags']:
              if pos not in pos_set:
                pos_set.append(pos)

        self.pos2idx['PAD'] = 0

        for idx, pos in enumerate(pos_set):
            self.pos2idx[pos] = len(self.pos2idx)

        for key, value in self.pos2idx.items():
            self.idx2pos[value] = key

        #For unknown pos assign noun, that is the most common, see link utili

        return

    #Alternative using text_tokenized.word_ids()
    def map_origin_word_to_bert(self, words, tokenizer):
        '''
        Create a mapping between all original words to the tokens generated by a transformer tokenizer
        '''
        bep_dict = {}
        current_idx = 0
        for word_idx, word in enumerate(words):
            bert_word = tokenizer.tokenize(word)
            word_len = len(bert_word)
            bep_dict[word_idx] = [current_idx, current_idx + word_len - 1]
            current_idx = current_idx + word_len
        return bep_dict

    def map_to_dict(self, N = None, predict = False) -> None:

      '''
      This method create for each sentence a mapping between the tokenized sentence and the original sentence.
      Since the tokenizer can split a word in multiple tokens, like clockwork in 'clock', '##work',
      I need a mapping between the position of a original word in the sub-words.

      Then for each token in lemmas, pos_tags, candidates and senses the relative vocabulary index are added
      if they are present else 'unk' index is added e.g. '1'
      '''


      self.data_set = [{} for i in range(len(self.sentences))]
      non_in_tags = 0

      for i, data in tqdm(enumerate(self.sentences[:N])):

        self.data_set[i]['words'] = data['words']
        self.data_set[i]['lemmas'] = []
        self.data_set[i]['pos'] = []
        self.data_set[i]['candidates'] = {}
        self.data_set[i]['original_candidates'] = self.sentences[i]['candidates']
        self.data_set[i]['glosses'] = self.sentences[i]['glosses']

        if predict is False:
            self.data_set[i]['senses'] = {}
            self.data_set[i]['original_senses'] = self.sentences[i]['senses']

        words_joined = ' '.join(data['words'])
        words_tokenized = tokenizer.tokenize(words_joined)

        bep_dict = self.map_origin_word_to_bert(data['words'], tokenizer)
        self.bep_dicts.append(bep_dict)

        for token in data['lemmas']:
          self.data_set[i]['lemmas'].append(self.lemmas2idx[token] if token in self.lemmas2idx else self.lemmas2idx['UNK'])

        for pos in data['pos_tags']:
          self.data_set[i]['pos'].append(self.pos2idx[pos])  #possible that i encounter a pos tag not in vocab

        for key, candidates in data['candidates'].items():
          for candidate in candidates:
            if key not in self.data_set[i]['candidates']:
              self.data_set[i]['candidates'][key] = []
            self.data_set[i]['candidates'][key].append(self.tags2idx[candidate] if candidate in self.tags2idx else self.tags2idx['UNK'])
            if candidate not in self.tags2idx:
              non_in_tags += 1

        if predict is False:
            for key, senses in data['senses'].items():
                for sense in senses:
                    if key not in self.data_set[i]['senses']:
                        self.data_set[i]['senses'][key] = []
                        self.data_set[i]['senses'][key].append(self.tags2idx[sense] if sense in self.tags2idx else self.tags2idx['UNK'])


        self.data_set[i]['len'] = len(words_tokenized)
      print(f'non in tags: {non_in_tags}')

      return

    def get_max_len(self):
      '''
        Return le length of the longest sentence in the dataset.
      '''
      longest = 0
      for i, elem in enumerate(self.data_set):
        if elem['len'] > longest:
          longest = elem['len']
      return longest

In [ ]:
class GlossaryDownloader():
    '''
      GlossaryDownloader is responsible to download the mapping between clusters and fine_grained.
      It also create and inverse mapping from the fine-grained to the clusters.
    '''
    def __init__(self):
        self.map_grained_to_fine = {}
        self.map_fine_to_grained = {}




    def downloadMapGrainedFine(self, DATASET_DIR: str) -> None:
        '''
          Download all the coarse to fine mapping, and create also an inverse map Dict
        '''
        data_path = os.path.join(DATASET_DIR, 'map', 'coarse_fine_defs_map.json')
        with open(data_path) as f:
            dictionary = f.read()
            dictionary = json.loads(dictionary)

            self.map_grained_to_fine = dictionary

        for sense_cluster, senses in self.map_grained_to_fine.items():
            for idx_sense, sense in enumerate(senses):
                for key, value in sense.items():
                    if key not in self.map_fine_to_grained:
                        self.map_fine_to_grained[key] = {
                            'cluster': sense_cluster,
                            'definition':  value
                        }
        return


In [ ]:
def get_max_len(data_set):
      '''
        Return le length of the longest sentence in the dataset.
      '''
      longest = 0
      for i, elem in enumerate(data_set):
        if elem['len'] > longest:
          longest = elem['len']
      return longest

# Dataset & DataLoader

In [ ]:
class DatasetWSD(Dataset):
  def __init__(self,
               dataset) -> None:

    self.data_set = dataset.data_set
    self.bep_dicts = dataset.bep_dicts

  ''' returns how many entries we have for a specific category '''
  def __len__(self) -> int:
    return len(self.data_set)

  ''' returns one item of one category '''
  def __getitem__(self, idx:int) -> dict:
    return self.data_set[idx], self.bep_dicts[idx]

In [ ]:
#fine_grained
def fine_collate_fn(d): #batch, bep_dicts
    '''
    Each sample of the batch contains the original sentence, original candidates, the labels and all the glosses.
    With original is meant before to be passed to a transformer tokenizer.
    The glosses contain all the sense definitions for each candidate sense.
    '''
      data_set = [elem[0] for elem in d]
      bep_dict = [elem[1] for elem in d]

      batch = []

      batches_tokens = []
      candidates_list = []
      glosses = []
      labels = []

      max_len = get_max_len(data_set)

      for i, data in enumerate(d):
        batch.append(copy.deepcopy(data_set[i]))

        batches_tokens.append(batch[i]['words'])
        candidates_list.append(batch[i]['original_candidates'])
        labels.append(batch[i]['original_senses'])
        glosses.append(batch[i]['glosses'])

      return batches_tokens, candidates_list, glosses, labels

In [ ]:
#coarse_grained
def collate_fn(d): #batch, bep_dicts

      '''
      Each sample of the batch contains:

      batches_tokens: the original sentence, batches_lemmas, , batches_pos, target_mask, candidate_mask, candidates_list, glosses
      batches_lemmas:  the lemmas mapped into their own dictionary,
      batches_pos: the pos tags mapped into their own dictionary,
      batches_labels: labels,
      target_mask: filter out only the correct sense, for each word to disambiguate(deprecated),
      candidate_mask: filter out only the candidate senses, for each word to disambiguate,
      candidates_list: The list of all candidate for each word to be disambiguated,
      glosses: all the sense definitions for each sense inside each candidate cluster.

      '''
      data_set = [elem[0] for elem in d]
      bep_dict = [elem[1] for elem in d]

      batch = []

      batches_tokens = []
      batches_lemmas = []
      batches_labels = []
      batches_pos = []
      target_mask = []
      candidates_list = []
      glosses = []

      max_len = get_max_len(data_set)

      for i, data in enumerate(d):
        batch.append(copy.deepcopy(data_set[i]))

        batches_tokens.append(batch[i]['words'])
        candidates_list.append(batch[i]['candidates'])
        glosses.append(batch[i]['glosses'])

        batch[i]['lemmas_mapped_bert'] = [lemmas2idx['PAD'] for i in range(max_len + 2)]
        batch[i]['pos_mapped_bert'] = [pos2idx['PAD'] for i in range(max_len + 2)]
        batch[i]['labels_mapped_bert'] = [tags2idx['PAD'] for i in range(max_len + 2)]
        batch[i]['target_mask'] = [0 for i in range(max_len + 2)]

        for idx, senses in batch[i]['senses'].items():

          idx_bep = bep_dict[i][int(idx)][0]
          batch[i]['labels_mapped_bert'][idx_bep + 1] = senses[0] #idx_bep + 1 because first and last elements must be PAD since are relative to CLS and SEP
          batch[i]['target_mask'][idx_bep + 1] = 1 #idx_bep + 1 because first and last elements must be PAD since are relative to CLS and SEP



        for idx, word in enumerate(batch[i]['words']):

          idx_bep_start = bep_dict[i][int(idx)][0]
          idx_bep_end = bep_dict[i][int(idx)][1]


          for j in range(idx_bep_start, idx_bep_end + 1):

            batch[i]['pos_mapped_bert'][j + 1] = batch[i]['pos'][idx] #j + 1 because first and last elements must be PAD since are relative to CLS and SEP
            batch[i]['lemmas_mapped_bert'][j + 1] = batch[i]['lemmas'][idx] #j + 1 because first and last elements must be PAD since are relative to CLS and SEP



        batches_lemmas.append(torch.LongTensor(batch[i]['lemmas_mapped_bert']))
        batches_labels.append(torch.LongTensor(batch[i]['labels_mapped_bert']))
        batches_pos.append(torch.LongTensor(batch[i]['pos_mapped_bert']))

        #create the candidate mask
        target_mask.append(torch.LongTensor(batch[i]['target_mask']).unsqueeze(1).repeat(1, len(tags2idx)))

      batches_lemmas = torch.stack(batches_lemmas)
      batches_labels = torch.stack(batches_labels)
      batches_pos = torch.stack(batches_pos)
      target_mask = torch.stack(target_mask)

      sizes = list(batches_labels.shape) + [len(tags2idx)]
      candidate_mask = torch.zeros(sizes)

      for i in range(candidate_mask.shape[0]):
        for idx, candidates in batch[i]['candidates'].items():
            for index, candidate in enumerate(candidates):
              print(candidate)
              idx_bep = bep_dict[i][int(idx)][0] + 1 # only the starting token, idx_bep + 1 because first and last elements must be PAD since are relative to CLS and SEP

              candidate_mask[i, idx_bep, candidate] = 1



      return batches_tokens, batches_lemmas, batches_labels, batches_pos, target_mask, candidate_mask, candidates_list, glosses

# Instation Dataset

## Instantion

In [ ]:
#Download the mapping between coarse and fine grained and create the inverse mapping
glossary = GlossaryDownloader()
glossary.downloadMapGrainedFine(DATASET_DIR)

In [ ]:
# vocabulary  ={
#     'lemmas2idx': lemmas2idx,
#     'tags2idx': tags2idx,
#     'pos2idx': pos2idx,
#     'idx2lemmas': idx2lemmas,
#     'idx2tags': idx2tags,
#     'idx2pos': idx2pos
# }

# path_file_json_coarse = "vocabulary_coarse.json"


# with open(path_file_json_coarse, 'w') as file_json:
#     json.dump(vocabulary, file_json)

In [ ]:
path_file_json_fine = "vocabulary_fine.json"
path_file_json_coarse = "vocabulary_coarse.json"

with open(path_file_json_fine, 'r') as file_json:
    vocabulary_fine = json.load(file_json)

with open(path_file_json_coarse, 'r') as file_json:
    vocabulary_coarse = json.load(file_json)

In [ ]:
#FINE-GRAINED DATASET
t_set = DatasetDownloader()
te_set = DatasetDownloader()
v_set = DatasetDownloader()

#Download all the datasets and remove all the sentences with a length grater than 60
t_set.set_data(DATASET_DIR, "fine-grained", "train_fine_grained", file_type_dir, 60) #nlp_2023/data/fine-grained/train_fine_grained.json
te_set.set_data(DATASET_DIR, "fine-grained", "test_fine_grained", file_type_dir, 60)
v_set.set_data(DATASET_DIR, "fine-grained", "val_fine_grained", file_type_dir, 60)

#set the glossary
t_set.setGlossary(glossary)
te_set.setGlossary(glossary)
v_set.setGlossary(glossary)

#get the glossary for all the datasets
t_set.downloadMapGrainedFine(DATASET_DIR)
t_set.retrieveGlossaryFromCandidates()

te_set.downloadMapGrainedFine(DATASET_DIR)
te_set.retrieveGlossaryFromCandidates()

v_set.downloadMapGrainedFine(DATASET_DIR)
v_set.retrieveGlossaryFromCandidates()

#Create a vocabolary for all the words in lemmas, candidate clusters and pos
# t_set.createVocabolary()
# t_set.createVocabolaryPos()

lemmas2idx_fine, tags2idx_fine, pos2idx_fine, idx2lemmas_fine, idx2tags_fine, idx2pos_fine = vocabulary_fine.values()

t_set.setDictionary(lemmas2idx_fine, tags2idx_fine, pos2idx_fine, idx2lemmas_fine, idx2tags_fine, idx2pos_fine)
te_set.setDictionary(lemmas2idx_fine, tags2idx_fine, pos2idx_fine, idx2lemmas_fine, idx2tags_fine, idx2pos_fine)
v_set.setDictionary(lemmas2idx_fine, tags2idx_fine, pos2idx_fine, idx2lemmas_fine, idx2tags_fine, idx2pos_fine)

#Generate the mapping between original words and the tokenized ones
t_set.map_to_dict()
te_set.map_to_dict()
v_set.map_to_dict()

train_set = DatasetWSD(t_set)
test_set = DatasetWSD(te_set)
val_set = DatasetWSD(v_set)

In [ ]:
glossary.map_grained_to_fine

In [ ]:
#COARSED-GRAINED DATASET

t_set_coarse = DatasetDownloader()
te_set_coarse = DatasetDownloader()
v_set_coarse = DatasetDownloader()

#Download all the datasets and remove all the sentences with a length grater than 60
t_set_coarse.set_data(DATASET_DIR, "coarse-grained", "train_coarse_grained", file_type_dir, 60) #nlp_2023/data/coarse-grained/train_coarse_grained.json
te_set_coarse.set_data(DATASET_DIR, "coarse-grained", "test_coarse_grained", file_type_dir, 60)
v_set_coarse.set_data(DATASET_DIR, "coarse-grained", "val_coarse_grained", file_type_dir, 60)

#set the glossary
t_set_coarse.setGlossary(glossary)
te_set_coarse.setGlossary(glossary)
v_set_coarse.setGlossary(glossary)

#get the glossary for all the datasets
# t_set_coarse.downloadMapGrainedFine(DATASET_DIR)
t_set_coarse.retrieveGlossaryFromSenses()

# te_set_coarse.downloadMapGrainedFine(DATASET_DIR)
te_set_coarse.retrieveGlossaryFromSenses()

# v_set_coarse.downloadMapGrainedFine(DATASET_DIR)
v_set_coarse.retrieveGlossaryFromSenses()

#Create a vocabolary for all the words in lemmas, candidate clusters and pos

# t_set_coarse.createVocabolary()
# t_set_coarse.createVocabolaryPos()
# lemmas2idx, tags2idx, pos2idx, idx2lemmas, idx2tags, idx2pos = t_set_coarse.getDictionary()

lemmas2idx, tags2idx, pos2idx, idx2lemmas, idx2tags, idx2pos = vocabulary_coarse.values()

# lemmas2idx = vocabulary_coarse['lemmas2idx']
# tags2idx = vocabulary_coarse['tags2idx']
# pos2idx = vocabulary_coarse['pos2idx']
# idx2lemmas = vocabulary_coarse['idx2lemmas']
# idx2tags = vocabulary_coarse['idx2tags']
# idx2pos = vocabulary_coarse['idx2pos']


t_set_coarse.setDictionary(lemmas2idx, tags2idx, pos2idx, idx2lemmas, idx2tags, idx2pos)
te_set_coarse.setDictionary(lemmas2idx, tags2idx, pos2idx, idx2lemmas, idx2tags, idx2pos)
v_set_coarse.setDictionary(lemmas2idx, tags2idx, pos2idx, idx2lemmas, idx2tags, idx2pos)

#Generate the mapping between original words and the tokenized ones
t_set_coarse.map_to_dict()
te_set_coarse.map_to_dict()
v_set_coarse.map_to_dict()

train_set_coarse = DatasetWSD(t_set_coarse)
test_set_coarse = DatasetWSD(te_set_coarse)
val_set_coarse = DatasetWSD(v_set_coarse)

In [ ]:
#FINE GRAINED

train_dataloader_fine = DataLoader(
    train_set,
    batch_size=8,
    shuffle=False,
    collate_fn=fine_collate_fn
 )

test_dataloader_fine = DataLoader(
    test_set,
    batch_size=8,
    shuffle=False,
    collate_fn=fine_collate_fn
 )

val_dataloader_fine = DataLoader(
    val_set,
    batch_size=8,
    shuffle=False,
    collate_fn=fine_collate_fn
 )

In [ ]:
#COARSE GRAINED

train_dataloader = DataLoader(
    train_set_coarse,
    batch_size=8,
    shuffle=False,
    collate_fn=collate_fn
 )

test_dataloader = DataLoader(
    test_set_coarse,
    batch_size=8,
    shuffle=False,
    collate_fn=collate_fn
 )

val_dataloader = DataLoader(
    val_set_coarse,
    batch_size=8,
    shuffle=False,
    collate_fn=collate_fn
 )

# WSDTrainer

## Fine-grained

In [ ]:
class WSDModuleFine():
    '''
      This class will handle the train loop.
    '''

    def __init__(self,
                hypers,
                model,
                optimizer,
                glossary,
                weights_path: str = None
                ) -> None:

        self.hypers = hypers
        self.model = model
        self.optimizer = optimizer

        self.glossary = glossary

        self.train_losses = []

        self.cluster_predictions = []
        self.cluster_labels = []

        if weights_path is not None:
            self.model.load_state_dict(torch.load(weights_path, map_location=torch.device(hypers.device)))

        return

    def train(self, dataloader, epochs = None):

        if epochs is None:
          epochs = self.hypers.epochs

        self.train_losses = []
        self.train_losses_epochs = []

        criterion = {}
        for key in self.glossary.map_fine_to_grained.keys():
	        criterion[key] = torch.nn.CrossEntropyLoss(reduction='none')

        self.model.train()

        for epoch in range(epochs):

            for i, data in tqdm(enumerate(dataloader), total = len(dataloader)):


                torch.cuda.empty_cache()

                batches_tokens, candidates_list, glosses, labels = data

                self.optimizer.zero_grad()

                loss, output = self.model(batches_tokens, glosses, criterion, candidates_list, labels)

                loss.backward()
                self.optimizer.step()

                self.train_losses.append(loss)



                del batches_tokens
                del candidates_list
                del glosses
                del labels

                torch.cuda.empty_cache()


                # accuracies.append(accuracy(output, labels).item())

                if i % 100 == 0:
                  print("iteration: ", i, " loss: ", loss.item())


            print("epoch: ", epoch, " loss: ", self.getLossesMean()['train'])

            self.train_losses_epochs.append(torch.tensor(self.train_losses).mean())


        print("End Training")
        return self.train_losses, self.train_losses_epochs

    def getPredictionLabels(self, output, candidate_mask, batches_labels = None):

      output = output.detach()
      batches_labels = batches_labels.detach()

      output = F.softmax(output, dim = -1)
      output = output * candidate_mask

      cluster_predictions = [[] for i in range(output.shape[0])]

      cluster_predictions_indices = output.argmax(-1).nonzero(as_tuple=True)

      tags_predicted = output.argmax(-1)[cluster_predictions_indices].tolist()

      # print(tup)
      for i, elem in enumerate(cluster_predictions_indices[0]):
        cluster_predictions[elem.item()].append(idx2tags[tags_predicted[i]])

      if batches_labels is not None:

        cluster_labels = [[] for i in range(output.shape[0])]
        cluster_labels_indices = batches_labels.nonzero(as_tuple=True)
        labels = batches_labels[cluster_labels_indices].tolist()


        for i, elem in enumerate(cluster_labels_indices[0]):
          cluster_labels[elem.item()].append(idx2tags[labels[i]])

        return cluster_predictions, cluster_labels

      return cluster_predictions



## Coarse-grained

In [ ]:
class WSDModuleGrained():
    '''
      This class will handle the train loop.
    '''

    def __init__(self,
                hypers,
                model,
                optimizer,
                idx2tags,
                weights_path: str = None
                ) -> None:

        self.hypers = hypers
        self.model = model
        self.optimizer = optimizer
        self.idx2tags = idx2tags

        self.train_losses = []

        self.cluster_predictions = []
        self.cluster_labels = []

        if weights_path is not None:
            self.model.load_state_dict(torch.load(weights_path, map_location=torch.device(hypers.device)))

        return

    def train(self, dataloader, epochs = None):

        if epochs is None:
          epochs = self.hypers.epochs

        self.train_losses = []
        self.train_losses_epochs = []

        self.model.train()

        for epoch in range(epochs):

            for i, data in tqdm(enumerate(dataloader), total = len(dataloader)):


                torch.cuda.empty_cache()

                batches_tokens, batches_lemmas, batches_labels, batches_pos, target_mask, candidate_mask, candidate_list, glosses = data

                batches_lemmas = batches_lemmas.to(hypers.device)
                batches_labels = batches_labels.to(hypers.device)
                batches_pos = batches_pos.to(hypers.device)
                target_mask = target_mask.to(hypers.device)
                candidate_mask = candidate_mask.to(hypers.device)

                self.optimizer.zero_grad()

                loss, output = self.model(batches_tokens, batches_lemmas, batches_pos, target_mask, batches_labels, glosses)

                loss.backward()
                self.optimizer.step()

                self.train_losses.append(loss)


                batches_lemmas.detach()
                batches_labels.detach()
                batches_pos.detach()
                target_mask.detach()
                candidate_mask.detach()

                del batches_lemmas
                del batches_labels
                del batches_pos
                del target_mask
                del candidate_mask
                del candidate_list
                del glosses

                # accuracies.append(accuracy(output, labels).item())

                if i % 100 == 0:
                  print("iteration: ", i, " loss: ", loss.item())


            print("epoch: ", epoch, " loss: ", self.getLossesMean()['train'])
            self.train_losses_epochs.append(torch.tensor(self.train_losses).mean())


        print("End Training")
        return self.train_losses, self.train_losses_epochs

    def getPredictionLabels(self, output, candidate_mask, batches_labels = None):

      output = output.detach()

      output = F.softmax(output, dim = -1)
      output = output * candidate_mask

      cluster_predictions = [[] for i in range(output.shape[0])]

      cluster_predictions_indices = output.argmax(-1).nonzero(as_tuple=True)

      tags_predicted = output.argmax(-1)[cluster_predictions_indices].tolist()

      # print(tup)
      for i, elem in enumerate(cluster_predictions_indices[0]):
        cluster_predictions[elem.item()].append(self.idx2tags[str(tags_predicted[i])])

      if batches_labels is not None:

        batches_labels = batches_labels.detach()


        cluster_labels = [[] for i in range(output.shape[0])]
        cluster_labels_indices = batches_labels.nonzero(as_tuple=True)
        labels = batches_labels[cluster_labels_indices].tolist()


        for i, elem in enumerate(cluster_labels_indices[0]):
          cluster_labels[elem.item()].append(self.idx2tags[str(labels[i])])

        return cluster_predictions, cluster_labels

      return cluster_predictions



# Hyper Parameters

In [ ]:
#Hyperparameters for the fine grained model
class hypers_fine:
    vocab_size = len(lemmas2idx_fine)

    vocab_pos_size = len(pos2idx_fine)

    embedding_dim = 768 #512

    number_of_tags = len(tags2idx_fine)

    input_size = 768

    learning_rate = 1e-4 #2e-4

    batch_size = 8

    epochs = 1

    dropout_rate = 0.25#0.3

    normal_loss_function = False

    bertForTokenClassificationLoss = True

    print_step = 100

    device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
#Hyperparameters for the coarse grained model
class hypers:
    vocab_size = len(lemmas2idx)

    vocab_pos_size = len(pos2idx)

    embedding_dim = 768 #512

    number_of_tags = len(tags2idx)

    input_size = 768

    learning_rate = 1e-4 #2e-4

    batch_size = 8

    epochs = 1

    dropout_rate = 0.25#0.3

    normal_loss_function = False

    bertForTokenClassificationLoss = True

    print_step = 100

    device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Model


## Fine Grained

In [ ]:
#Gloss WSD on Fine-grained dataset

class GlossWSDFine(nn.Module):

    def __init__(self, hypers):
        super(GlossWSDFine, self).__init__()

        self.tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
        self.model_bert_context = AutoModel.from_pretrained("bert-base-cased")
        self.model_bert_gloss = AutoModel.from_pretrained("bert-base-cased")

        self.embedding_lemmas = nn.Embedding(hypers.vocab_size + 1, hypers.embedding_dim, padding_idx = lemmas2idx['PAD'])
        self.embedding_pos = nn.Embedding(hypers.vocab_pos_size + 1, hypers.embedding_dim, padding_idx = pos2idx['PAD'])

        self.layer_norm = nn.LayerNorm(hypers.embedding_dim)

        self.classifier = nn.Linear(hypers.embedding_dim, hypers.number_of_tags)

        self.dropout = nn.Dropout(hypers.dropout_rate)

        self.relu = nn.ReLU()

        self.hypers = hypers

    def forward(self, batches_tokens, glosses = None, criterion = None, candidates_list = None, labels = None):

        x = tokenizer(batches_tokens, return_tensors="pt",
                                  padding='longest',
                                  is_split_into_words=True).to(hypers.device)

        attention_mask = x['attention_mask'].squeeze(1).to(hypers.device)
        input_ids = x['input_ids'].squeeze(1).to(hypers.device)

        outputs = self.model_bert_context(input_ids,
                            attention_mask=attention_mask)

        sequence_output = outputs[0]


        loss = 0
        total_length = 0
        predictions = []
        for b_g_idx, b_g in enumerate(glosses): #b_g = batch_gloss
            prediction_batch = []
            for index_gloss, gloss in b_g.items():
                gloss_arr = [[g] for g in gloss]
                gloss_embed = tokenizer(gloss_arr, return_tensors="pt",
                                padding='longest',
                                is_split_into_words=True).to(hypers.device)

                outputs_gloss = self.model_bert_gloss(**gloss_embed).last_hidden_state #senses X max_len X 768


                cls_embeddings = outputs_gloss[:, 0, :] #senses X 768

                word_ids = tokenizer(batches_tokens[b_g_idx], return_tensors="pt",
                                    padding='longest',
                                    is_split_into_words=True).word_ids()

                word_indexes = np.where(np.array(word_ids) == int(index_gloss))
                word_candidate = sequence_output[b_g_idx][word_indexes].mean(dim=0).unsqueeze(0)

                mm = torch.mm(word_candidate, cls_embeddings.T)

                if labels is not None:

                    label_name = labels[b_g_idx][index_gloss][0]

                    label_idx = candidates_list[b_g_idx][index_gloss].index(label_name)

                    label = torch.tensor([label_idx]).to(hypers.device)

                    loss += criterion[label_name](mm, label)
                    total_length += 1

                    # free RAM and GPU RAM
                    label.detach()

                    del label_idx
                    del label_name
                    del label

                else:
                    prediction_batch.append(candidates_list[b_g_idx][index_gloss][mm.argmax()])

                outputs_gloss.detach()
                cls_embeddings.detach()
                mm.detach()
                word_candidate.detach()

                del outputs_gloss
                del cls_embeddings
                del mm
                del word_candidate

            predictions.append(prediction_batch)



        if labels is not None:

            loss = loss / total_length

            outputs = (loss, outputs)

            return outputs  # (loss), scores, (hidden_states), (attentions)

        else:
            return predictions




## Coarse Grained

In [ ]:
#Incorporate other staff

class GlossWSDGrained(nn.Module):

    def __init__(self, hypers):
        super(GlossWSDGrained, self).__init__()

        self.tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
        self.bert_context = AutoModel.from_pretrained("bert-base-cased")
        self.bert_gloss = AutoModel.from_pretrained("bert-base-cased")

        self.embedding_lemmas = nn.Embedding(hypers.vocab_size + 1, hypers.embedding_dim, padding_idx = lemmas2idx['PAD'])
        self.embedding_pos = nn.Embedding(hypers.vocab_pos_size + 1, hypers.embedding_dim, padding_idx = pos2idx['PAD'])

        self.layer_norm = nn.LayerNorm(hypers.embedding_dim)

        #fc layer transforms the output to give the final output layer

        # self.fc1 = nn.Linear(hypers.embedding_dim * 3, hypers.embedding_dim * 2)
        self.fc1 = nn.Linear(hypers.embedding_dim, hypers.embedding_dim * 2)

        self.fc2 = nn.Linear(hypers.embedding_dim * 2, hypers.embedding_dim * 1)

        self.classifier = nn.Linear(hypers.embedding_dim, hypers.number_of_tags)

        self.dropout = nn.Dropout(hypers.dropout_rate)

        self.relu = nn.ReLU()

        self.hypers = hypers

    def forward(self, batches_tokens, lemmas = None, pos = None, target_mask = None, labels = None, glosses = None):

        # lemmas = self.dropout(self.embedding_lemmas(lemmas))   # dim: batch_size x batch_max_len x embedding_dim
        # pos = self.dropout(self.embedding_pos(pos))

        # batch_size x batch_max_len x embedding_dim ->
        # batch_size x batch_max_len x 1(50) x embedding_dim
        # batch_size x batch_max_len x 50 x embedding_dim

        x = self.tokenizer(batches_tokens, return_tensors="pt",
                                  padding='longest',
                                  is_split_into_words=True).to(hypers.device)

        attention_mask = x['attention_mask'].squeeze(1).to(hypers.device)
        input_ids = x['input_ids'].squeeze(1).to(hypers.device)

        outputs = self.bert_context(input_ids,
                            attention_mask=attention_mask)

        sequence_output = outputs[0]

        if glosses is not None:
            for b_g_idx, b_g in enumerate(glosses): #b_g = batch_gloss
                for index_gloss, gloss in b_g.items():
                    gloss_arr = [[g] for g in gloss]
                    gloss_embed = self.tokenizer(gloss_arr, return_tensors="pt",
                                    padding='longest',
                                    is_split_into_words=True).to(hypers.device)

                    outputs_gloss = self.bert_gloss(**gloss_embed).last_hidden_state #senses X max_len X 768

                    cls_embeddings = outputs_gloss[:, 0, :] #senses X 768

                    word_candidate = sequence_output[b_g_idx, int(index_gloss) + 1, :].unsqueeze(0).detach().cpu().numpy() # 1 X 768

                    dot_prod = np.dot(word_candidate, cls_embeddings.detach().cpu().numpy().T)
                    best_sense_idx = dot_prod.argmax()

                    #substitute the embedding of a word to disambguate adding to it the embedding of the most similar sense
                    sequence_output[b_g_idx, best_sense_idx] = (sequence_output[b_g_idx, best_sense_idx] + cls_embeddings[best_sense_idx,:]) / 2

                    outputs_gloss.detach()
                    cls_embeddings.detach()

                    del outputs_gloss
                    del cls_embeddings
                    del word_candidate
                    del dot_prod
                    del best_sense_idx

        # concat = torch.cat((sequence_output, lemmas, pos), dim = -1) #batch X words_len X embedding_dim * 3

        concat = self.dropout(self.relu(self.fc1(sequence_output))) #batch X words_len X embedding_dim * 2
        concat = self.dropout(self.relu(self.fc2(concat))) #batch X words_len X embedding_dim

        logits = self.classifier(concat)

        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here


        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            # Only keep active parts of the loss

            if attention_mask is not None:
                active_loss = attention_mask.view(-1) == 1
                active_logits = logits.view(-1, hypers.number_of_tags)[active_loss]
                active_labels = labels.view(-1)[active_loss]


                loss = loss_fct(active_logits, active_labels)
            else:
                loss = loss_fct(logits.view(-1, hypers.number_of_tags), labels.view(-1))



            outputs = (loss,) + outputs

        return outputs  # (loss), scores, (hidden_states), (attentions)




In [ ]:
# # WORKING VERSION

class BaselineWSD(nn.Module):

    def __init__(self, hypers):
        super(BaselineWSD, self).__init__()

        self.tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
        self.bert = AutoModel.from_pretrained("bert-base-cased")

        self.embedding_lemmas = nn.Embedding(hypers.vocab_size + 1, hypers.embedding_dim, padding_idx = lemmas2idx['PAD'])
        self.embedding_pos = nn.Embedding(hypers.vocab_pos_size + 1, hypers.embedding_dim, padding_idx = pos2idx['PAD'])

        self.layer_norm = nn.LayerNorm(hypers.embedding_dim)

        self.fc1 = nn.Linear(hypers.embedding_dim * 3, hypers.embedding_dim * 1)
        self.fc2 = nn.Linear(hypers.embedding_dim * 2, hypers.embedding_dim * 1) #Kept to match the parameters found in training and saved but it is not used
        self.classifier = nn.Linear(hypers.embedding_dim, hypers.number_of_tags)

        self.dropout = nn.Dropout(hypers.dropout_rate)

        self.relu = nn.ReLU()

        self.hypers = hypers

    def forward(self, batches_tokens, lemmas, pos, target_mask, labels = None):

        lemmas = self.dropout(self.embedding_lemmas(lemmas))   # dim: batch_size x batch_max_len x embedding_dim
        pos = self.dropout(self.embedding_pos(pos))

        # batch_size x batch_max_len x embedding_dim ->
        # batch_size x batch_max_len x 1(50) x embedding_dim
        # batch_size x batch_max_len x 50 x embedding_dim

        x = tokenizer(batches_tokens, return_tensors="pt",
                                  padding='longest',
                                  is_split_into_words=True).to(hypers.device)

        attention_mask = x['attention_mask'].squeeze(1).to(hypers.device)
        input_ids = x['input_ids'].squeeze(1).to(hypers.device)

        outputs = self.bert(input_ids,
                            attention_mask=attention_mask)

        sequence_output = outputs[0]

        concat = torch.cat((sequence_output, lemmas, pos), dim = -1) #batch X words_len X embedding_dim * 3
        concat = self.dropout(self.relu(self.fc1(concat))) #batch X words_len X embedding_dim

        logits = self.classifier(concat)

        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            # Only keep active parts of the loss

            if attention_mask is not None:
                active_loss = attention_mask.view(-1) == 1
                active_logits = logits.view(-1, hypers.number_of_tags)[active_loss]
                active_labels = labels.view(-1)[active_loss]


                loss = loss_fct(active_logits, active_labels)
            else:
                loss = loss_fct(logits.view(-1, hypers.number_of_tags), labels.view(-1))



            outputs = (loss,) + outputs

        return outputs  # (loss), scores, (hidden_states), (attentions)




# Train and test

## Train

In [ ]:
model_fine = GlossWSDFine(hypers_fine)
model_fine.to(hypers_fine.device)

optimizer = torch.optim.Adam(model_fine.parameters(), lr=hypers_fine.learning_rate)

wsd_handler_fine = WSDModuleFine(hypers_fine, model_fine, optimizer, "drive/MyDrive/AI/nlp_model_hw2_gloss_fine_epoch_2.pth")

In [ ]:
it_losses, epoch_losses = wsd_handler_fine.train(train_dataloader, 1)

In [ ]:
model = BaselineWSD(hypers)
model.to(hypers.device)

optimizer = torch.optim.Adam(model.parameters(), lr=hypers.learning_rate)

wsd_handler_coarse = WSDModuleGrained(hypers, model, optimizer, idx2tags)

In [ ]:
it_losses, epoch_losses = wsd_handler_coarse.train(train_dataloader, 1)

## Aux Functions

In [ ]:
def compute_accuracy(predictions: List[List[str]], labels: List[List[str]]) -> Tuple[int, int, int]:
  '''
  Compute the accuracy.
  '''
  all_accuracies = []
  total_accuracy = 0
  total_counter = 0

  for batch_index in range(len(predictions)):
    accuracy = 0
    counter = 0
    for sen_pred, sen_lab in zip(predictions[batch_index], labels[batch_index]):
      for word_pred, word_lab in zip(sen_pred, sen_lab):
        counter += 1
        total_counter += 1
        if word_pred in word_lab:
          accuracy += 1
          total_accuracy += 1

    all_accuracies.append(accuracy / counter)
  return all_accuracies, total_accuracy, total_counter

def filter_candidate_given_coarse_pred(candidates_list: List[Dict], glosses: List[Dict], pred_coarse: List[List[str]]) -> Tuple[List[Dict], List[Dict]]:
  '''
  Given the coarse-grained prediction from the coarse-grained model,
  filters the possible fine-grained candidates to the just the ones belonging to that cluster.

  Return the list of new sense candidates for fine-grained with their relative definitions.

  Example of candidate_list:
  [{'52': ['knot.n.h.01', 'mil.n.h.03', 'mi.n.h.04', 'mi.n.h.06'],
  '69': ['us.n.h.02']},
  {'24': ['stay.v.h.02', 'rest.v.h.02', 'stay.v.h.01']}]
  '''
  new_candidate_list = []
  new_glosses = []

  for id_sample, sample in enumerate(candidates_list):
      new_candidates = {}
      new_gloss = {}

      for f, (id_candidates, candidates) in enumerate(sample.items()):
          new_candidate = []
          new_gls = []
          # print(candidates)

          for i, candidate in enumerate(candidates):

              if glossary.map_fine_to_grained[candidate]['cluster'] in pred_coarse[id_sample][f] or pred_coarse[id_sample][f] == 'UNK':
                  # print(f'cluster: {pred_coarse[id_sample][f]}, candidate: {candidate}')

                  new_candidate.append(candidate)
                  new_gls.append(glosses[id_sample][id_candidates][i])

          new_candidates[id_candidates] = new_candidate
          new_gloss[id_candidates] = new_gls

      new_candidate_list.append(new_candidates)
      new_glosses.append(new_gloss)

  return new_candidate_list, new_glosses

def dict_to_arr_lab(labels: List[Dict]) -> List[List[List[str]]]:
  '''
  Convert a candidate list from List[Dict] to List[List[List[str]]]

  Example:
  From:
  [{'52': ['knot.n.h.01', 'mil.n.h.03', 'mi.n.h.04', 'mi.n.h.06'],
  '69': ['us.n.h.02']},
  {'24': ['stay.v.h.02', 'rest.v.h.02', 'stay.v.h.01']}]

  To:
  [[['knot.n.h.01', 'mil.n.h.03', 'mi.n.h.04', 'mi.n.h.06'], ['us.n.h.02']],
  [['stay.v.h.02', 'rest.v.h.02', 'stay.v.h.01']]]
  '''

  labels_arr = [ [word[0] for word in list(sample.values())] for sample in labels]
  return labels_arr

def retrieve_cluster_given_senses(predictions: List[List[str]]) -> List[List[str]]:
  '''
  Given the senses predicted by the fine-grained return the cluster to which they belong.
  '''
  predictions_grained = []
  for pred_batch in predictions:
      predictions_batch_grained = []
      for p in pred_batch:
          predictions_batch_grained.append(glossary.map_fine_to_grained[p]['cluster'])

      predictions_grained.append(predictions_batch_grained)

  return predictions_grained

## Tests

In [ ]:
criterion = {}
for key in t_set.map_fine_to_grained.keys():
    criterion[key] = torch.nn.CrossEntropyLoss(reduction='none')

In [ ]:
# model_bert_context = AutoModel.from_pretrained("bert-base-cased").to(hypers.device)
# model_bert_gloss = AutoModel.from_pretrained("bert-base-cased").to(hypers.device)

### Test on fine-grained model

In [ ]:
# model = GlossWSD(hypers)
model_fine = GlossWSDFine(hypers_fine)
model_fine.to(hypers_fine.device)

model_fine.load_state_dict(torch.load("drive/MyDrive/AI/nlp_model_hw2_gloss_fine_epoch_2.pth", map_location=torch.device(hypers.device)))

In [ ]:

model_fine.eval()

dataiter_fine = iter(train_dataloader_fine)
# dataiter_coarse = iter(train_dataloader)


all_predictions_fine = []
all_labels_fine = []

for index_batch in tqdm(range(len(train_dataloader_fine)), total = len(train_dataloader_fine)):
  data_fine = next(dataiter_fine)

  with torch.no_grad():

      batches_tokens, candidates_list, glosses, labels = data_fine
      predictions = model_fine(batches_tokens, glosses, criterion, candidates_list)

      all_predictions_fine.append(predictions)
      all_labels_fine.append(labels)

      # if index_batch == 1:
      #   break

all_labels_fine_arr = [dict_to_arr_lab(l) for l in all_labels_fine]

accuracies, total_acc, total_counter = compute_accuracy(all_predictions_fine, all_labels_fine_arr)
print(f'accuracy: {total_acc/total_counter}, loss: {1-(total_acc/total_counter)}')


### Test on corase_grained model

In [ ]:
model = GlossWSDGrained(hypers)
# model = BaselineWSD(hypers)
model.to(hypers.device)
model.load_state_dict(torch.load("drive/MyDrive/AI/nlp_model_hw2_gloss_coarse_epoch_7.pth", map_location=torch.device(hypers.device)))


In [ ]:
model.eval()

dataiter = iter(test_dataloader)

all_predictions = []
all_labels = []

for i in tqdm(range(len(test_dataloader)), total = len(test_dataloader)):
  data = next(dataiter)

  with torch.no_grad():

      batches_tokens, batches_lemmas, batches_labels, batches_pos, target_mask, candidate_mask, candidates_list, glosses = data

      batches_lemmas = batches_lemmas.to(hypers.device)
      batches_labels = batches_labels.to(hypers.device)
      batches_pos = batches_pos.to(hypers.device)
      target_mask = target_mask.to(hypers.device)
      candidate_mask = candidate_mask.to(hypers.device)

      #add UNK as possible result
      il = batches_labels.nonzero(as_tuple=True) + (torch.tensor([1]), )
      candidate_mask[il] = 1

      loss, output = model(batches_tokens, batches_lemmas, batches_pos, target_mask, batches_labels)
      # output = model(batches_tokens, batches_lemmas, batches_pos, target_mask)

      pred_coarse, labels_coarse = wsd_handler_coarse.getPredictionLabels(output, candidate_mask, batches_labels)

      all_predictions.append(pred_coarse)
      all_labels.append(labels_coarse)


accuracies, total_acc, total_counter = compute_accuracy(all_predictions, all_labels)
print(f'accuracy: {total_acc/total_counter}, loss: {1-(total_acc/total_counter)}')




### Test before coarse, then fine(prediction on fine dataset)
I use the corase grained model to predict the correct cluster from which i extract all the sense candidates. On these candidates i disambiguate the correct sense with the model trained on the fine-grained dataset.

In [ ]:
dataiter_fine = iter(test_dataloader_fine)
dataiter_coarse = iter(test_dataloader)


all_predictions_fine = []
all_labels_fine = []

all_predictions_coarse = []
all_labels_coarse = []

for index_batch in tqdm(range(len(test_dataloader_fine)), total = len(test_dataloader_fine)):
  data_fine = next(dataiter_fine)
  data_coarse = next(dataiter_coarse)

  with torch.no_grad():

      batches_tokens, batches_lemmas, batches_labels, batches_pos, target_mask, candidate_mask, candidates_list, glosses = data_coarse

      batches_lemmas = batches_lemmas.to(hypers.device)
      batches_labels = batches_labels.to(hypers.device)
      batches_pos = batches_pos.to(hypers.device)
      target_mask = target_mask.to(hypers.device)
      candidate_mask = candidate_mask.to(hypers.device)

      il = batches_labels.nonzero(as_tuple=True) + (torch.tensor([1]), )
      candidate_mask[il] = 1

      # predictions = model(batches_tokens, batches_lemmas, batches_pos, target_mask, batches_labels, glosses, criterion, candidates_list)
      loss, output = model(batches_tokens, batches_lemmas, batches_pos, target_mask, batches_labels)
      pred_coarse, labels_coarse = wsd_handler_coarse.getPredictionLabels(output, candidate_mask, batches_labels)

      all_predictions_coarse.append(pred_coarse)
      all_labels_coarse.append(labels_coarse)

      batches_tokens, candidates_list, glosses, labels = data_fine

      new_candidates_list, new_glosses = filter_candidate_given_coarse_pred(candidates_list, glosses, pred_coarse)

      predictions = model_fine(batches_tokens, new_glosses, criterion, new_candidates_list)

      all_predictions_fine.append(predictions)
      all_labels_fine.append(labels)

      # if index_batch == 1:
      #   break


100%|██████████| 77/77 [00:16<00:00,  4.80it/s]


In [ ]:
all_labels_fine_arr = [dict_to_arr_lab(l) for l in all_labels_fine]

accuracies, total_acc, total_counter = compute_accuracy(all_predictions_fine, all_labels_fine_arr)
print(f'accuracy: {total_acc/total_counter}')

accuracies, total_acc, total_counter = compute_accuracy(all_predictions_coarse, all_labels_coarse)
print(f'accuracy: {total_acc/total_counter}')

accuracies mean: 0.7999563037760133
accuracy: 0.801025641025641
accuracies mean: 0.9194186861012681
accuracy: 0.918974358974359


### Test Before fine, then extract the coarse cluster(prediction on coarse dataset)

Here i predict with the model trained in the fine-grained and then extract the cluster to chich the senses belongs to.

In [ ]:
dataiter_fine = iter(test_dataloader_fine)
dataiter_coarse = iter(test_dataloader)


all_predictions_fine = []
all_labels_fine = []

all_labels_coarse = []


for index_batch in tqdm(range(len(test_dataloader_fine)), total = len(test_dataloader_fine)):
  data_fine = next(dataiter_fine)
  data_coarse = next(dataiter_coarse)

  with torch.no_grad():

      batches_tokens, candidates_list, glosses, labels = data_fine
      predictions = model_fine(batches_tokens, glosses, criterion, candidates_list)

      all_predictions_fine.append(predictions)
      all_labels_fine.append(labels)

      batches_tokens, batches_lemmas, batches_labels, batches_pos, target_mask, candidate_mask, candidates_list, glosses = data_coarse

      cluster_labels = [[] for i in range(len(batches_tokens))]
      cluster_labels_indices = batches_labels.nonzero(as_tuple=True)
      labels = batches_labels[cluster_labels_indices].tolist()


      for i, elem in enumerate(cluster_labels_indices[0]):
        cluster_labels[elem.item()].append(idx2tags[str(labels[i])])

      all_labels_coarse.append(cluster_labels)



In [ ]:
all_predictions_fine_arr = [elem for batches in all_predictions_fine for elem in batches ]
all_labels_fine_arr = [elem for batches in all_labels_fine for elem in batches ]
all_labels_coarse_arr = [elem for batches in all_labels_coarse for elem in batches ]
labels_arr = dict_to_arr_lab(all_labels_fine_arr)


In [ ]:
# print(compute_accuracy([all_predictions_fine_arr], [labels_arr]))
print(compute_accuracy([retrieve_cluster_given_senses(all_predictions_fine_arr)], [retrieve_cluster_given_senses(labels_arr)]))